In [ ]:
!pip3 install numpy>=1.20 matplotlib==3.7.5 pandas scikit-learn imbalanced-learn

/bin/bash: /home/wytcor/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [142]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Carregamento do dataset
dataset = pd.read_csv(filepath_or_buffer="/home/wytcor/PROJECTs/mestrado-ufes/lab-life/datasets/zr7vgbcyr2-1/metadata.csv", sep=",")
dataset


,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,...,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,...,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,...,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2293,PAT_1708,3156,NaN,NaN,NaN,NaN,73,NaN,NaN,NaN,...,NaN,ACK,True,False,False,False,False,False,PAT_1708_3156_175.png,False
2294,PAT_46,880,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,...,12.0,BCC,True,True,False,True,False,False,PAT_46_880_140.png,True
2295,PAT_1343,1217,NaN,NaN,NaN,NaN,74,NaN,NaN,NaN,...,NaN,SEK,False,False,False,False,False,False,PAT_1343_1217_404.png,False
2296,PAT_326,690,False,False,POMERANIA,POMERANIA,58,True,FEMALE,True,...,4.0,BCC,True,False,False,False,False,True,PAT_326_690_823.png,True


Remover os dados que são inválidos

In [143]:
dataset=dataset.dropna()
dataset=dataset.drop(columns=['lesion_id', 'patient_id', 'img_id'])
## Descomentar se for usar a moda das variáveis como entrada
#dataset.fillna(dataset.mode().iloc[0], inplace=True)
#dataset.replace([True, False], [1, 0])

Separação entre os dados das features e dos labels

Uso do hot-encoder para converter as variáveis categóricas para inteiras

In [144]:
from sklearn.preprocessing import OneHotEncoder

# Seleção das features
dataset_features = dataset.drop(columns=['diagnostic'])  # Features

# Identificar variáveis categóricas
categorical_cols = dataset_features.select_dtypes(include=['object']).columns

# Criar o OneHotEncoder
ohe = OneHotEncoder(sparse_output=False)  # Use 'drop="first"' para evitar multicolinearidade

# Aplicar o OneHotEncoder às colunas categóricas
categorical_data = ohe.fit_transform(dataset_features[categorical_cols])

# Criar um DataFrame para os dados one-hot codificados
categorical_df = pd.DataFrame(categorical_data, columns=ohe.get_feature_names_out(categorical_cols), index=dataset_features.index)

# Combinar os dados transformados com as demais features
dataset_features = dataset_features.drop(columns=categorical_cols)  # Remove as colunas categóricas originais
dataset_features = pd.concat([dataset_features, categorical_df], axis=1)

dataset_features = dataset_features.replace([True, False], [1, 0])
# Verificar os primeiros valores após a transformação
print(dataset_features)


      age  fitspatrick  diameter_1  diameter_2  biopsed  smoke_False  \
1      55          3.0         6.0         5.0        1          1.0   
4      79          1.0         5.0         5.0        1          1.0   
6      52          3.0        15.0        10.0        1          1.0   
7      74          1.0        15.0        10.0        1          1.0   
9      58          1.0         9.0         7.0        0          1.0   
...   ...          ...         ...         ...      ...          ...   
2289   66          2.0        20.0        15.0        1          0.0   
2291   41          2.0         9.0         5.0        1          1.0   
2292   43          2.0         9.0         5.0        1          1.0   
2294   55          3.0        13.0        12.0        1          1.0   
2296   58          3.0         5.0         4.0        1          1.0   

      smoke_True  drink_False  drink_True  background_father_AUSTRIA  ...  \
1            0.0          1.0         0.0                 

/tmp/ipykernel_7152/582628473.py:22: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset_features = dataset_features.replace([True, False], [1, 0])


In [145]:
from sklearn.preprocessing import LabelEncoder

# Inicialize o LabelEncoder
le = LabelEncoder()

y = dataset['diagnostic']  # label

# Fit e transform os rótulos em X_train e y_train
y_encoded = le.fit_transform(y)
y_encoded

array([1, 1, 1, ..., 4, 1, 1])

Caso seja necessário usar o RandomUndersampling

In [146]:
from imblearn.under_sampling import RandomUnderSampler

# Inicialize o RandomUnderSampler
rus = RandomUnderSampler(random_state=42)

# Aplique a subamostragem
dataset_features_undersampled, y_encoded_undersampled = rus.fit_resample(dataset_features, y_encoded)

# Verifique o número de amostras por classe após o balanceamento
from collections import Counter
print("Distribuição original:", Counter(y_encoded))
print("Distribuição após subamostragem:", Counter(y_encoded_undersampled))


Distribuição original: Counter({1: 845, 0: 273, 4: 192, 3: 72, 2: 52, 5: 40})
Distribuição após subamostragem: Counter({0: 40, 1: 40, 2: 40, 3: 40, 4: 40, 5: 40})


In [147]:

# Dividir dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(dataset_features_undersampled, y_encoded_undersampled, test_size=0.2, random_state=42, shuffle=True)

Uso do modelo de bayers para verificar o impacto da feature no modelo

In [148]:

# Criar o modelo Random Forest
model = RandomForestClassifier(n_estimators=25, random_state=0, max_depth=10)

# Treinar o modelo
model.fit(X_train, y_train)

# Verificar a acurácia no conjunto de teste
accuracy = model.score(X_test, y_test)
print(f'Acurácia: {accuracy * 100:.2f}%')

Acurácia: 58.33%


Análise da importância das features     

In [149]:
import pandas as pd

# Get feature importances
feature_importances = model.feature_importances_

# Map feature importance to feature names
features = X_train.columns
importance_df = pd.DataFrame({'Feature': features, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display top features
print(importance_df.head(10))


            Feature  Importance
0               age    0.086702
3        diameter_2    0.068654
2        diameter_1    0.066336
4           biopsed    0.047155
60        itch_True    0.041206
59       itch_False    0.033246
74  elevation_False    0.030093
75   elevation_True    0.028738
63        grew_True    0.027807
68    changed_False    0.026033


In [150]:
features_used

['age',
 'fitspatrick',
 'diameter_1',
 'diameter_2',
 'biopsed',
 'smoke_True',
 'drink_True',
 'background_father_BRASIL',
 'background_father_BRAZIL',
 'background_father_CZECH',
 'background_father_GERMANY',
 'background_father_ISRAEL',
 'background_father_ITALY',
 'background_father_NETHERLANDS',
 'background_father_POLAND',
 'background_father_POMERANIA',
 'background_father_PORTUGAL',
 'background_father_SPAIN',
 'background_father_UNK',
 'background_mother_FRANCE',
 'background_mother_GERMANY',
 'background_mother_ITALY',
 'background_mother_NETHERLANDS',
 'background_mother_NORWAY',
 'background_mother_POLAND',
 'background_mother_POMERANIA',
 'background_mother_PORTUGAL',
 'background_mother_SPAIN',
 'background_mother_UNK',
 'pesticide_True',
 'gender_MALE',
 'skin_cancer_history_True',
 'cancer_history_True',
 'has_piped_water_True',
 'has_sewage_system_True',
 'region_ARM',
 'region_BACK',
 'region_CHEST',
 'region_EAR',
 'region_FACE',
 'region_FOOT',
 'region_FOREARM',
 

Usando um modelo de regressão logistica para obter a importância de cada feature

In [151]:
from sklearn.linear_model import LogisticRegression

# Logistic regression coefficients
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

# Get coefficients
coefficients = model.coef_[0]
importance_df = pd.DataFrame({'Feature': features, 'Coefficient': coefficients})
importance_df = importance_df.sort_values(by='Coefficient', ascending=False)

print(importance_df.head(10))


                      Feature  Coefficient
62                 grew_False     1.011802
68              changed_False     0.942360
23   background_mother_FRANCE     0.686974
52             region_FOREARM     0.667656
61                   itch_UNK     0.625739
21      background_father_UNK     0.602746
54                 region_LIP     0.574296
37  skin_cancer_history_False     0.541940
45             region_ABDOMEN     0.461768
24  background_mother_GERMANY     0.392529


/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Avaliação do uso de permutação das features no impacto da acurácia do modelo

In [152]:
from sklearn.inspection import permutation_importance

# Permutation importance
perm_importance = permutation_importance(model, X_test, y_test, scoring='accuracy')

# Map results to features
perm_importance_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Importance': perm_importance.importances_mean
})
perm_importance_df = perm_importance_df.sort_values(by='Importance', ascending=False)

print(perm_importance_df.head(10))


          Feature  Importance
0             age    0.141667
3      diameter_2    0.108333
2      diameter_1    0.100000
68  changed_False    0.087500
4         biopsed    0.083333
62     grew_False    0.070833
1     fitspatrick    0.062500
63      grew_True    0.054167
59     itch_False    0.045833
70    changed_UNK    0.041667


Uso do Naive Bayes para verificar o impacto final de cada variável sobre  modelo

Analisando uma feature por vez no treino, e outras sendo adicionadas com o tempo


In [ ]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, roc_auc_score
from sklearn.naive_bayes import GaussianNB
# Modelo Naive Bayes
model = GaussianNB()

# Lista para armazenar os resultados
results = []
features_used = []

# Incrementalmente incluir features
for i in range(1, X_train.shape[1] + 1):
    # Selecionar as primeiras `i` features
    X_train_subset = X_train[:, :i] if isinstance(X_train, np.ndarray) else X_train.iloc[:, :i]
    X_test_subset = X_test[:, :i] if isinstance(X_test, np.ndarray) else X_test.iloc[:, :i]
    
    # Treinar o modelo com as features selecionadas
    model.fit(X_train_subset, y_train)
    
    # Obter as probabilidades de saída para o conjunto de teste
    probas = model.predict_proba(X_test_subset)
    # Verificando a estrutura das probabilidades
    # print(probas[:,1])

    # Predições do modelo
    predictions = model.predict(X_test_subset)
    
    # Calcular métricas (roc_auc_score usa as probabilidades da classe positiva)
    roc_auc = roc_auc_score(y_test, probas, multi_class='ovr', average='macro') if len(np.unique(y_test)) > 2 else roc_auc_score(y_test, probas[:, 1])
    
    # Fetures usadas
    features_used.append(features[i-1])
    # Armazenar resultados
    results.append({
        'features_used': features_used,  # Índices das features usadas
        'probabilities': probas,
        'predictions': predictions,
        'true_labels': y_test,
        'accuracy_score': accuracy_score(y_test, predictions),
        'balanced_accuracy_score': balanced_accuracy_score(y_test, predictions),
        'f1_score': f1_score(y_test, predictions, average='weighted'), 
        'precision_score': precision_score(y_test, predictions, average='weighted'),
        'roc_auc_score': roc_auc
    })


['age']
['age', 'fitspatrick']
['age', 'fitspatrick', 'diameter_1']
['age', 'fitspatrick', 'diameter_1', 'diameter_2']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA', 'background_father_BRASIL']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_F

/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA', 'background_father_BRASIL', 'background_father_BRAZIL', 'background_father_CZECH', 'background_father_GERMANY', 'background_father_ISRAEL', 'background_father_ITALY', 'background_father_NETHERLANDS', 'background_father_POLAND', 'background_father_POMERANIA', 'background_father_PORTUGAL', 'background_father_SPAIN', 'background_father_UNK', 'background_mother_BRAZIL', 'background_mother_FRANCE', 'background_mother_GERMANY', 'background_mother_ITALY', 'background_mother_NETHERLANDS', 'background_mother_NORWAY', 'background_mother_POLAND']
['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA', 'background_father_BRASIL', 'background_father_BRAZIL', 'background_father_CZECH', 'background_father_GERMANY', 'background_father_ISRAEL', 'background_father_I

/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA', 'background_father_BRASIL', 'background_father_BRAZIL', 'background_father_CZECH', 'background_father_GERMANY', 'background_father_ISRAEL', 'background_father_ITALY', 'background_father_NETHERLANDS', 'background_father_POLAND', 'background_father_POMERANIA', 'background_father_PORTUGAL', 'background_father_SPAIN', 'background_father_UNK', 'background_mother_BRAZIL', 'background_mother_FRANCE', 'background_mother_GERMANY', 'background_mother_ITALY', 'background_mother_NETHERLANDS', 'background_mother_NORWAY', 'background_mother_POLAND', 'background_mother_POMERANIA', 'background_mother_PORTUGAL', 'background_mother_SPAIN', 'background_mother_UNK', 'pesticide_False', 'pesticide_True', 'gender_FEMALE', 'gender_MALE', 'skin_cancer_history_False', 'skin_cancer_history_True', 'cancer_history_False', 'cancer_history_True', 'has_piped_water_False'

/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

['age', 'fitspatrick', 'diameter_1', 'diameter_2', 'biopsed', 'smoke_False', 'smoke_True', 'drink_False', 'drink_True', 'background_father_AUSTRIA', 'background_father_BRASIL', 'background_father_BRAZIL', 'background_father_CZECH', 'background_father_GERMANY', 'background_father_ISRAEL', 'background_father_ITALY', 'background_father_NETHERLANDS', 'background_father_POLAND', 'background_father_POMERANIA', 'background_father_PORTUGAL', 'background_father_SPAIN', 'background_father_UNK', 'background_mother_BRAZIL', 'background_mother_FRANCE', 'background_mother_GERMANY', 'background_mother_ITALY', 'background_mother_NETHERLANDS', 'background_mother_NORWAY', 'background_mother_POLAND', 'background_mother_POMERANIA', 'background_mother_PORTUGAL', 'background_mother_SPAIN', 'background_mother_UNK', 'pesticide_False', 'pesticide_True', 'gender_FEMALE', 'gender_MALE', 'skin_cancer_history_False', 'skin_cancer_history_True', 'cancer_history_False', 'cancer_history_True', 'has_piped_water_False'

/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/wytcor/miniconda3/envs/eda/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

Exibir os dados da filtragem 

In [154]:

# Exibir os resultados
for result in results:
    print(f"accuracy_score usadas: {result['accuracy_score']}")
    print(f"balanced_accuracy_score:\n{result['balanced_accuracy_score']}")
    print(f"f1_score usadas: {result['f1_score']}")
    print(f"precision_score:\n{result['precision_score']}")
    print(f"roc_auc_score:\n{result['roc_auc_score']}")

    print("-" * 40)


accuracy_score usadas: 0.1875
balanced_accuracy_score:
0.19477513227513224
f1_score usadas: 0.16431751727804358
precision_score:
0.14777352277352276
roc_auc_score:
0.6147688620902484
----------------------------------------
accuracy_score usadas: 0.22916666666666666
balanced_accuracy_score:
0.23941798941798945
f1_score usadas: 0.20877525252525253
precision_score:
0.22414529914529915
roc_auc_score:
0.6208883548782691
----------------------------------------
accuracy_score usadas: 0.25
balanced_accuracy_score:
0.24822631072631074
f1_score usadas: 0.23983395989974934
precision_score:
0.24036195286195286
roc_auc_score:
0.6317680300610675
----------------------------------------
accuracy_score usadas: 0.25
balanced_accuracy_score:
0.25562169312169314
f1_score usadas: 0.22005639645442276
precision_score:
0.20234718984718988
roc_auc_score:
0.6462902866901732
----------------------------------------
accuracy_score usadas: 0.20833333333333334
balanced_accuracy_score:
0.21052188552188555
f1_scor

Exportar os dados em um formato de arquivo csv

In [155]:
import pandas as pd

def save_results_to_csv(results, filename='model_results.csv'):
    # Criação de uma lista para armazenar os dados formatados para o CSV
    rows = []
    
    # Itera sobre os resultados e organiza as métricas e as features usadas
    for result in results:
        row = {
            'features_used': result['features_used'],
            'accuracy_score': result['accuracy_score'],
            'balanced_accuracy_score': result['balanced_accuracy_score'],
            'f1_score': result['f1_score'],
            'precision_score': result['precision_score'],
            'roc_auc_score': result['roc_auc_score']
        }
        rows.append(row)
    
    # Convertendo a lista de resultados para um DataFrame
    df = pd.DataFrame(rows)
    
    # Salvando o DataFrame em um arquivo CSV
    df.to_csv(filename, index=False)

In [156]:
save_results_to_csv(results, './results/naive-bayes/model_results_with_undersampling.csv')